# Prompt Claude with Batching

This notebook is for prompting claude with a batch of inputs. It takes up to 24h for the results to come in.

This first cell is setup and should always be executed.

In [2]:
%load_ext autoreload
%autoreload 2

import anthropic
from dotenv import load_dotenv
import os
import datetime
import json
from pathlib import Path
from src import objects, functions_claude2, metadata
from setup import output_struct

load_dotenv()
api_key = os.getenv("API_KEY_CLAUDE")
client = anthropic.Anthropic(api_key=api_key)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Load the data used to evaluate the model.

Load the prompt.

In [ ]:
data_dir = Path('/home/brunobrocai/Data/MoWiKo/Paper-themKorp-2/final_corpus')
data = []
for file in data_dir.glob('*.json'):
    with open(file, 'r', encoding='utf-8') as f:
        data.append(json.load(f))

data = [d for d in data if d.get('Claude_0shot', None) is None]
len(data)

In [6]:
# SETUP SETTINGS
prompt_name = 'zero_shot'
prompt_file = 'setup/prompts.yaml'

output_format = output_struct.TextClassification

temperature = 0.0

model = 'claude-3-7-sonnet-20250219'

In [7]:
prompt = objects.SysUserPrompt()
prompt.load_from_file(
    prompt_file,
    prompt_name,
)

In [ ]:
# cost calculation
import tiktoken

text_count = len(data)
full_text = ''
for entry in data:
    full_text += entry['text_deduped'] + '\n'

# Make sure it is a validly encoded string
full_text = full_text.encode('utf-8', errors='ignore').decode('utf-8', errors='ignore')

enc = tiktoken.encoding_for_model("gpt-4o")
decoded = enc.encode(full_text)
tokens = len(decoded)

cost = functions_claude2.PricingSonnet.input_tokens_batch * tokens
cost = cost + len(enc.encode(prompt.get_sysprompt())) * text_count * functions_claude2.PricingSonnet.input_tokens_batch
cost = cost + len(enc.encode(prompt.get_userprompt(''))) * text_count * functions_claude2.PricingSonnet.input_tokens_batch
cost

Create the prompt batch and submit it.

In [ ]:
run_metadata = metadata.RunMetadata(
    run_id=datetime.datetime.now().strftime("%Y%m%d%H%M%S"),
    model=model,
    time=datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    prompt_name=prompt_name,
    prompt=str(prompt),
    output_format=output_format['name'],
    dataset='final',
    seed=0,
    temperature=temperature,
)
batch_filedir = 'batches/claude'
batch = functions_claude2.batch_requests(
    data, prompt, client, output_format,
    batch_filedir=batch_filedir,
    run_info=run_metadata,
    tool_coice={'type': 'tool', 'name': output_format['name']},
    # drop_instances=already_seen,
    temperature=temperature,
    model=model,
)

Now, WAIT...

After at most 24h, your responses should be ready. Check in the online console and download manually, and process with 'eval_claude'.